<a href="https://colab.research.google.com/github/41371125h-chinrouzhen/114-1-PL/blob/main/HW2_%E6%88%90%E7%B8%BE%E4%B8%80%E6%9C%AC%E9%80%9A%EF%BC%88AI%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **成績一本通**
目標：從 Sheet 讀各次作業/測驗 → 計平時/期末/總成績 → 回寫成績總表。

AI 點子：針對「開放式作答」欄位，用模型產出整體摘要與常見迷思整理（不評分，只做總結）。

Google sheet鏈接:https://docs.google.com/spreadsheets/d/1HA1cvniLmO1O362GtHlrLEK3TQ8_XQJBULMpbaU-29s/edit?usp=sharing

In [3]:
# 安裝 匯入套件與認證
!pip install -q gradio gspread pandas google-generativeai

import pandas as pd
import gspread
from google.colab import auth, userdata
from google.auth import default
import google.generativeai as genai
import gradio as gr

In [15]:
# Google Sheets
SHEET_URL = "https://docs.google.com/spreadsheets/d/1HA1cvniLmO1O362GtHlrLEK3TQ8_XQJBULMpbaU-29s/edit?usp=sharing"

gc = None
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    print("✅ Google Sheets 認證成功。")
except Exception as e:
    print(f"❌ Google Sheets 認證失敗: {e}")

✅ Google Sheets 認證成功。


In [16]:
# Gemini API 認證 (使用 Colab Secrets)
model = None
try:
    api_key = userdata.get('GEMINI_API_KEY')
    if not api_key:
        raise ValueError("在 Colab Secrets 中找不到 'GEMINI_API_KEY'。")
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel('gemini-2.5-flash')
    print("✅ Gemini API 已成功設定。")

    # 執行一個簡單的測試來確認 API 是否正常運作
    response = model.generate_content('Hi')
    if response.text:
      print("👍 Gemini API 測試連線成功。")

except Exception as e:
    print(f"❌ Gemini API 設定或測試失敗: {e}")

✅ Gemini API 已成功設定。
👍 Gemini API 測試連線成功。


In [19]:
# 功能函式
# Cell 3: 定義核心功能函式

def add_record(student_id, name, hw1, hw2, hw3, q1, q2, final, open_response):
    """將一筆新的學生紀錄新增到 Google Sheet"""
    if not gc:
        return "錯誤：Google Sheets 未認證。"
    try:
        spreadsheet = gc.open_by_url(SHEET_URL)
        worksheet = spreadsheet.sheet1

        # 準備要新增的資料列
        new_row = [student_id, name, hw1, hw2, hw3, q1, q2, final, open_response]
        worksheet.append_row(new_row)

        print(f"成功新增紀錄：{new_row}")
        return f"✅ 成功新增學號 {student_id} 的紀錄！"
    except Exception as e:
        print(f"新增紀錄時發生錯誤: {e}")
        return f"❌ 新增紀錄失敗: {e}"

def get_class_analysis():
    """讀取所有學生的資料，並產生全班的 AI 分析報告"""
    if not gc or not model:
        return "錯誤：認證未完成，無法產生報告。", None

    try:
        # 讀取資料
        spreadsheet = gc.open_by_url(SHEET_URL)
        worksheet = spreadsheet.sheet1
        data = worksheet.get_all_records()
        if not data:
            return "報告：工作表中沒有資料可供分析。", pd.DataFrame()

        df = pd.DataFrame(data)

        # --- AI 全班分析 ---
        prompt = f"""
        你是一位專業的程式設計課程助教。請根據整個班級的成績資料，提供一份總結報告。

        全班成績資料：
        {df.to_string()}

        請依據以下結構產出報告：
        1.  **整體表現摘要**：總結全班的平均表現、分數分布情況（例如：整體表現平均、但高低分差距大）。指出哪些評量項目（作業、小測、期末）對學生來說挑戰較大。
        2.  **常見學習迷思整理**：根據「開放式作答」欄位的內容，以及學生在各項作業/小測成績的普遍表現，推測學生可能存在的 2-3 個常見學習迷思或困難點。
        3.  **給老師的教學建議**：基於上述分析，提供 1-2 點具體的教學調整建議，例如：可以多加強哪個單元的說明，或是在課堂上多帶一些練習。

        """

        print("正在為全班產生 AI 分析報告...")
        response = model.generate_content(prompt)
        ai_report = response.text
        print("✅ 全班分析報告已產生。")

        # 同時回傳報告和完整的 DataFrame
        return ai_report, df

    except Exception as e:
        print(f"產生分析報告時發生錯誤: {e}")
        return f"❌ 產生報告失敗: {e}", None

print("✅ 核心功能函式已成功定義。")

✅ 核心功能函式已成功定義。


In [20]:
# Gradio 建立

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 👨‍🏫 AI 教學助理與成績分析平台")

    # 使用 Tab 元件來區分不同功能
    with gr.Tabs():
        # 第一個分頁：新增學生資料
        with gr.TabItem("✍️ 新增學生資料"):
            gr.Markdown("請在下方欄位輸入單筆學生資料，然後點擊按鈕新增至 Google Sheet。")
            with gr.Row():
                student_id = gr.Textbox(label="學號")
                name = gr.Textbox(label="姓名")
            with gr.Row():
                hw1 = gr.Number(label="作業1", value=0)
                hw2 = gr.Number(label="作業2", value=0)
                hw3 = gr.Number(label="作業3", value=0)
            with gr.Row():
                q1 = gr.Number(label="小測1", value=0)
                q2 = gr.Number(label="小測2", value=0)
                final = gr.Number(label="期末", value=0)
            open_response = gr.Textbox(label="開放式作答", lines=3)

            add_button = gr.Button("新增紀錄", variant="primary")
            add_status = gr.Markdown() # 用於顯示新增成功或失敗的訊息

            add_button.click(
                fn=add_record,
                inputs=[student_id, name, hw1, hw2, hw3, q1, q2, final, open_response],
                outputs=add_status
            )

        # 第二個分頁：全班成績分析
        with gr.TabItem("📊 全班成績分析"):
            gr.Markdown("點擊按鈕以讀取 Google Sheet 中的所有資料，並產生全班的 AI 分析報告。")
            analyze_button = gr.Button("讀取資料並產生分析報告", variant="primary")

            gr.Markdown("### 📜 全班成績總覽")
            class_dataframe = gr.DataFrame(label="所有學生資料")

            gr.Markdown("### 🤖 AI 總結報告")
            class_report = gr.Markdown(label="AI 分析與教學建議")

            analyze_button.click(
                fn=get_class_analysis,
                inputs=[],
                outputs=[class_report, class_dataframe]
            )

print("Gradio 介面準備啟動...")
demo.launch(debug=True)

Gradio 介面準備啟動...
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://be45c605642930512a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


正在為全班產生 AI 分析報告...
✅ 全班分析報告已產生。
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://be45c605642930512a.gradio.live
